<a href="https://colab.research.google.com/github/saidurgaprasad4518/My-first-project/blob/main/Dashboard_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import os

def generate_visualizations():
    """
    Connects to the database, analyzes data, and generates multiple PNG files
    for key performance indicators.
    """
    try:
        # --- 1. Connect to the database ---
        conn = sqlite3.connect('company_database.db')
        print("Successfully connected to the database.")

        # --- 2. Load and process data ---
        products_df = pd.read_sql_query("SELECT * FROM products", conn)
        sales_df = pd.read_sql_query("SELECT * FROM sales", conn)

        sales_df['sale_date'] = pd.to_datetime(sales_df['sale_date'])

        merged_df = pd.merge(sales_df, products_df, on='product_id')
        merged_df['total_revenue'] = merged_df['quantity'] * merged_df['price']

        # --- 3. Set up the visualization style ---
        sns.set_style("whitegrid")
        plt.style.use('seaborn-v0_8-deep')

        output_dir = 'dashboard_visuals'
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        print("Generating visualizations...")

        # --- Plot 1: Monthly Revenue Trend ---
        plt.figure(figsize=(12, 7))
        merged_df['month'] = merged_df['sale_date'].dt.to_period('M')
        monthly_revenue = merged_df.groupby('month')['total_revenue'].sum()
        monthly_revenue.index = monthly_revenue.index.astype(str)

        sns.lineplot(x=monthly_revenue.index, y=monthly_revenue.values, marker='o', color='#3498db', linewidth=3)
        plt.title('Monthly Revenue Trend', fontsize=20, fontweight='bold', pad=20)
        plt.xlabel('Month', fontsize=14)
        plt.ylabel('Total Revenue ($)', fontsize=14)
        plt.xticks(rotation=45, ha='right')
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, '1_monthly_revenue_trend.png'))
        print("✓ Created '1_monthly_revenue_trend.png'")
        plt.close()

        # --- Plot 2: Revenue by Product Category ---
        plt.figure(figsize=(10, 8))
        category_sales = merged_df.groupby('category')['total_revenue'].sum().sort_values(ascending=False)
        sns.barplot(x=category_sales.values, y=category_sales.index, palette='Spectral', orient='h')
        plt.title('Total Revenue by Product Category', fontsize=20, fontweight='bold', pad=20)
        plt.xlabel('Total Revenue ($)', fontsize=14)
        plt.ylabel('Product Category', fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, '2_revenue_by_category.png'))
        print("✓ Created '2_revenue_by_category.png'")
        plt.close()

        # --- Plot 3: Units Sold by Product ---
        plt.figure(figsize=(10, 8))
        product_sales = merged_df.groupby('product_name')['quantity'].sum().sort_values(ascending=False)
        sns.barplot(x=product_sales.values, y=product_sales.index, palette='YlGnBu', orient='h')
        plt.title('Total Units Sold by Product', fontsize=20, fontweight='bold', pad=20)
        plt.xlabel('Total Units Sold', fontsize=14)
        plt.ylabel('Product Name', fontsize=14)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, '3_units_sold_by_product.png'))
        print("✓ Created '3_units_sold_by_product.png'")
        plt.close()

        # --- Plot 4: Sales by Customer Country (Pie Chart) ---
        plt.figure(figsize=(9, 9))
        country_sales = merged_df.groupby('customer_country')['total_revenue'].sum()
        plt.pie(country_sales, labels=country_sales.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("coolwarm"))
        plt.title('Sales Distribution by Customer Country', fontsize=20, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, '4_sales_by_country.png'))
        print("✓ Created '4_sales_by_country.png'")
        plt.close()

        # --- Plot 5: Relationship between Price and Quantity Sold ---
        plt.figure(figsize=(10, 7))
        sns.scatterplot(x='price', y='quantity', data=merged_df, hue='category', palette='viridis', s=100, alpha=0.7)
        plt.title('Price vs. Quantity Sold by Product Category', fontsize=20, fontweight='bold', pad=20)
        plt.xlabel('Price ($)', fontsize=14)
        plt.ylabel('Quantity Sold', fontsize=14)
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, '5_price_vs_quantity.png'))
        print("✓ Created '5_price_vs_quantity.png'")
        plt.close()

        print("\nAll visualizations have been successfully generated and saved in the 'dashboard_visuals' directory.")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if 'conn' in locals() and conn:
            conn.close()
            print("Database connection closed.")

if __name__ == "__main__":
    generate_visualizations()